In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
import keras
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPool2D, BatchNormalization, Dropout

import tensorflow as tf
import librosa
import os
import glob
import matplotlib.pyplot as plt

In [ ]:
def createSpectrogram(path):
  y, sr = librosa.load(path , duration=2.97)
  spect = librosa.feature.melspectrogram(y=y, sr=sr)
  return spect

In [ ]:
filename = '../input/gtzan-dataset-music-genre-classification/Data/genres_original'
gen_dict = dict()

data = np.zeros((999, 128, 128)).astype(np.float64)
# labels = np.empty((1000, 1)).astype(np.int32)
labels = []
label_to_num = {k:v for v, k in enumerate(os.listdir(filename))}
num_to_label = {v:k for v, k in enumerate(os.listdir(filename))}

for folder in os.listdir(filename):
    new = filename + "/" + folder + "/*.wav"
    for row, wav_file in enumerate(glob.glob(new)):
        if wav_file != '../input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00054.wav':
            spect = createSpectrogram(wav_file)
            data[row, :, :] = spect
            labels.append(label_to_num[folder])


In [ ]:
# labels = np.array(labels)
# np.save('labels.npy', labels)
np.save('data.npy', data)

# print(labels)

In [ ]:
# labelss = tf.keras.utils.to_categorical(labels , 10)
# np.save('spectrograms.npy' , data)
# np.save('labels.npy' , labels)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2,random_state=42, shuffle = True)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5,random_state=42, shuffle = True)

def get_label(vec, num_to_label):
    index = np.argmax(vec)
    return num_to_label[index]


In [ ]:
def add_dim_label(arr):
    shape = arr.shape
    return np.array([elm.reshape(shape[1], 1) for elm in arr])
# y_train_add = add_dim_label(y_train)
# y_test_add = add_dim_label(y_test)
# y_val_add = add_dim_label(y_val)

def add_dim(arr):
    shape = arr.shape
    return np.array([elm.reshape(shape[1], shape[2], 1) for elm in arr])

X_train_add = add_dim(X_train)
X_test_add = add_dim(X_test)
X_val_add = add_dim(X_val)

print("X_train final shape:", X_train_add.shape)
print("X_test final shape:", X_test_add.shape)
print("X_val final shape:", X_val_add.shape)


In [ ]:
input_shape = X_train_add.shape
def build_model(input_shape):
    model = Sequential([
        Conv2D(256, kernel_size = (3,3), padding = 'same', activation = 'relu'),
        MaxPool2D(pool_size = (3,3)),
        BatchNormalization(),
        Dropout(0.4),
        
        Conv2D(128, kernel_size = (3,3), padding = 'same', activation = 'relu'),
        MaxPool2D(pool_size = (4,4)),
        BatchNormalization(),
        Dropout(0.4),
        
        Conv2D(64, kernel_size = (4,4), strides = 2, padding = 'same',  activation = 'relu'),
        MaxPool2D(pool_size = (4,4)),
        BatchNormalization(),
        Dropout(0.4),
        
        Flatten(),
        Dense(100, activation = 'relu'),
        Dense(10, activation = 'softmax')
        
    ])
    model.compile( loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
cnn_model = build_model(input_shape)
cnn_model.fit(X_train_add, y_train, epochs = 150, verbose = 1, batch_size = 10, validation_data = (X_val_add, y_val))

In [ ]:
cnn_model.save('zeby.h5')

In [ ]:
# cnn_model.evaluate(X_val_add, y_val_add)
pred = cnn_model.predict(X_val_add)
preds = [np.argmax(i) for i in pred]
# print( y_train.squeeze())

# def get_preds(model, path,num_to_label):
#     spect = createSpectrogram(path)
#     spect = spect.reshape((1, spect.shape[0], spect.shape[1], 1))
#     predi = np.argmax(model.predict(spect))
#     return num_to_label[predi]
# wav_path = '../input/gtzan-dataset-music-genre-classification/Data/genres_original/classical/*.wav'
# for wav_file in glob.glob(wav_path):
#     print(get_preds(cnn_model, wav_file, num_to_label))


# Split_test_train
# labels to one_hot_encoded
# Model => CNN
# Model => LSTM



In [ ]:
gen_dict['jazz'].append(gen_dict['jazz'][0])
for key in gen_dict.keys():
    print(key, len(gen_dict[key]))

In [ ]:
data_key_dict = dict()
for key, value in gen_dict.items():
    data_ser = pd.Series(value)
    key_ser = pd.Series([key for i in range(100)])
    data_key_dict[key] = [key_ser, data_ser]

In [ ]:
total_data = data_key_dict['blues'][1]
tot_label = data_key_dict['blues'][0]
for key, value in data_key_dict.items():
    if key != 'blues':
        total_data = total_data.append(value[1], ignore_index = True)
        tot_label = tot_label.append(value[0], ignore_index = True)

In [ ]:
tot_data_df = total_data.to_frame()
# tot_data_df['label'] = tot_label
tot_data_df.to_csv('my_data.csv')

In [ ]:
path = '../input/gtzan-dataset-music-genre-classification/Data/features_30_sec.csv'



file = pd.read_csv(path)
file = file.drop(['filename'], axis = 1)
y = file.label
X = file.drop(columns = ['label'])
print(y)
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
dummy_y = np_utils.to_categorical(encoded_y)
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, encoded_y, 
                                                    test_size=0.2, random_state=0, shuffle=True)

def baseline_model():
# create model
    model = Sequential()
    model.add(Input(shape = (None, 58)))
    model.add(Conv1D(64, kernel_size=(3), activation='relu', input_shape = (58, 1)))
    model.add(Conv1D(128, kernel_size=(3), activation='relu'))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))# Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=1)
estimator = baseline_model()
print(estimator.summary())
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
train_X = tf.reshape(tf.convert_to_tensor(train_X), shape = (800, 58))
test_y = tf.reshape(tf.convert_to_tensor(train_y), shape = (800, 1))

data = tf.data.Dataset.from_tensor_slices((train_X, train_y))

estimator.fit(data, epochs = 200, batch_size=10, verbose = 1)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))